In [ ]:
!pip install openai
import openai
openai.api_key = 'OpenAIAPI金鑰'
!pip install gradio
import gradio as gr
import json

GPT_MODEL = 'gpt-3.5-turbo'

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",              # 函式名稱
      "description": "取得指定位置的當前天氣",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {                           # 參數名稱
            "type": "string",                     # 資料型別
            "description": "城市名稱",             # 參數說明
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    },
  }
]

In [ ]:
def get_current_weather(location, unit="fahrenheit"):
  if "tokyo" in location.lower():
    return json.dumps({"location":"Tokyo",
              "temperature":"19","unit":unit})
  elif "taipei" in location.lower():
    return json.dumps({"location":"Taipei",
              "temperature":"32","unit":unit})
  elif "paris" in location.lower():
    return json.dumps({"location":"Paris",
              "temperature":"22","unit":unit})
  else:
    return json.dumps({"location":location,
              "temperature":"unknown"})

In [ ]:
def run_conversation(query):
  messages = [{"role": "user", "content": query}]
  response = openai.chat.completions.create(
    model = GPT_MODEL,
    messages = messages,
    tools = tools,
    tool_choice = "auto",
  )
  response_message = response.choices[0].message
  tool_calls = response_message.tool_calls
  if tool_calls:
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    messages.append(response_message)
    for tool_call in tool_calls:
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
      function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
      )
      messages.append(
        {
          "tool_call_id": tool_call.id,
          "role": "tool",
          "name": function_name,
          "content": function_response,
        }
      )
    second_response = openai.chat.completions.create(
      model=GPT_MODEL,
      messages=messages,
    )
    return second_response.choices[0].message.content
  return response.choices[0].message.content

In [ ]:
gr.Interface(
    fn = run_conversation,
    inputs = gr.Textbox(label="輸入您的問題："),
    outputs = gr.Textbox(label="AI 的回答："),
    title = "AI 聊天室",
    allow_flagging = "never"
).queue().launch()